<h3>Genetic Algorithm</h3>

Genetic algorithms simulate the process of natural selection [ is a mechanism of evolution that explains how populations of living organisms adapt and change over time. ]. In simpler words, they simulate 'survival of the fittest' among individuals of consecutive generations to solve a problem.

<h5>Libraries and variables</h5>

In [29]:
import random
import math

from tqdm import trange,tqdm #visualizing the loops

GLOBAL_CHROMOSOME_LENGTH=4
GLOBAL_RANDINT_MIN=0
GLOBAL_RANDINT_MAX=8
GLOBAL_POP_LIMIT=16             #it is a X-X grid

GLOBAL_MUTATOR_CHANCE=100
GLOBAL_MUTATOR_RANGE=1000

GLOBAL_EPOCH_COUNT=4
GLOBAL_FITTING_DATA=[i for i in range(GLOBAL_CHROMOSOME_LENGTH)]

<h5>Gene Class Creation</h5>

In [30]:
class Gene:
    chromosomes=[]
    saved_chromosome_length=0
    saved_randint_min=0
    saved_randint_max=0
    fitness_factor=9999
    def __init__(self,fitting_data=GLOBAL_FITTING_DATA,input_list=[],
                 chromosome_length=GLOBAL_CHROMOSOME_LENGTH,randint_min=GLOBAL_RANDINT_MIN,randint_max=GLOBAL_RANDINT_MAX,
                 gene_generator='random',evaluator='default-evaluator',initial_eval=False): #constructor
        self.chromosomes=[]
        self.saved_chromosome_length=chromosome_length
        self.saved_randint_min=randint_min
        self.saved_randint_max=randint_max
        match gene_generator: 
            case 'none':
                pass
            case 'random': #initializes the gene based on chromosome length and randint range
                for i in range(chromosome_length):
                    self.chromosomes.append(random.randint(randint_min,randint_max)) 
            case 'input':  #takes in an input list and assigns the list to the chromosomes and updates the chromosome length to match the input list
                self.chromosomes=input_list
                self.saved_chromosome_length=len(input_list)
        if initial_eval:
            try:
                self.fitness_evaluation(fitting_data,evaluator=evaluator)
            except:
                pass
    def crossover_with_gene(self,input_gene, #crosses over with another gene, uniform crossover, includes mutation
                            crossover_method='cartesian-prod-random-uniform',
                            mutate=True): 
        child=Gene(chromosome_length=self.saved_chromosome_length,gene_generator='')
        match crossover_method:
            case 'cartesian-prod-random-uniform': #50-50 gamba on every chromosome 
                for i in range(self.saved_chromosome_length):
                    if bool(random.getrandbits(1)):
                        child.chromosomes.append(self.chromosomes[i])
                    else:
                        child.chromosomes.append(input_gene.chromosomes[i])
            case 'equate-cartesian-prod-random-uniform':
                pass
            case 'average-integer':
                for i in range(self.saved_chromosome_length):
                    child.chromosomes.append(int((self.chromosomes[i]+input_gene.chromosomes[i])/2))
        child.random_mutator(mutate=mutate)
        return child
    def random_equation_go_brrr(self,input_gene, #BRRRRRR
                                equation_target='minimize'): 
        equated_value=0
        match equation_target:
            case 'minimize':
                
                int(self.chromosomes[0])-int()

    def random_mutator(self,mutate=True): #imitates mutation based on mutation chances 
        if mutate:
            if random.randint(1,GLOBAL_MUTATOR_RANGE)<=GLOBAL_MUTATOR_CHANCE:
                self.chromosomes[random.randint(0,self.saved_chromosome_length-1)]=random.randint(self.saved_randint_min,self.saved_randint_max)
    def fitness_evaluation(self,fitting_data, #generates the fitness factor based on fitting_data
                           evaluator='default-evaluator'): 
        match evaluator:
            case 'default-evaluator':
                self.fitness_factor=self.saved_chromosome_length
                for i in range(self.saved_chromosome_length):
                    if self.chromosomes[i]==fitting_data[i]:
                        self.fitness_factor-=1
        return self.fitness_factor
    def display(self): #returns a string with the chromosomes and fitness factor as output
        return str('Chromosomes: '+str(self.chromosomes)+' Fitness Factor: '+str(self.fitness_factor))


<h5>Gene List Class</h5>
<p>It is a 2D Square [AxA] matrix comprised of 'Gene's

In [31]:
class Gene_List: 
    #Initialization of Stored Variables
    stored_epoch_count=0
    stored_pop_limit=0
    stored_chromosome_length=0
    stored_randint_min=0
    stored_randint_max=0
    stored_generation_count=0
    
    #Initialization of Preferred Settings
    preferred_gene_generator='random'
    preferred_crossover_method='cartesian-prod-random-uniform'
    preferred_crossover_prune_method='random-survivor'
    preferred_evaluator='default-evaluator'

    #Initialization of Boolean Variables
    bool_stop=True
    bool_verbose=False
    bool_mutate=True
    bool_initial_eval=False
    bool_manual=False

    bool_pcheck_safe=True
    #Initialization of Internal Lists
    fitting_data=[]
    generation_list=[]
    index_list=[]
    gene_list=[]
    
    def __init__(self,fitting_data=GLOBAL_FITTING_DATA,epoch=GLOBAL_EPOCH_COUNT,pop_limit=GLOBAL_POP_LIMIT,
                 chromosome_length=GLOBAL_CHROMOSOME_LENGTH,randint_min=GLOBAL_RANDINT_MIN,randint_max=GLOBAL_RANDINT_MAX,
                 gene_generator=preferred_gene_generator,crossover_method=preferred_crossover_method,crossover_prune_method=preferred_crossover_prune_method,evaluator=preferred_evaluator,
                 stop=bool_stop,verbose=bool_verbose,mutate=bool_mutate,initial_eval=bool_initial_eval,manual=bool_manual):
        #Store Input Parameters
        self.stored_epoch_count=epoch
        self.stored_pop_limit=pop_limit
        self.stored_chromosome_length=chromosome_length
        self.stored_randint_min=randint_min
        self.stored_randint_max=randint_max

        self.preferred_gene_generator=gene_generator
        self.preferred_crossover_method=crossover_method
        self.preferred_crossover_prune_method=crossover_prune_method
        self.preferred_evaluator=evaluator

        self.bool_stop=stop
        self.bool_verbose=verbose
        self.bool_mutate=mutate
        self.bool_initial_eval=initial_eval
        self.bool_manual=manual

        self.fitting_data=fitting_data

        #Emptying Generated Lists
        self.generation_list=[]
        self.index_list=[]

        self.Parameter_Checks()
        if self.bool_pcheck_safe:
            self.gene_list=self.Gene_List_Gen(pop_limit=pop_limit,                      #generate initial gene list [Square]
                                            chromosome_length=chromosome_length,randint_min=randint_min,randint_max=randint_max,
                                            gene_generator=gene_generator,evaluator=evaluator,
                                            initial_eval=initial_eval)             
            self.generation_list.append(self.gene_list)     #appends the initial gene list to the generation list, it is generation zero
            if manual==False:
                self.List_Iterator()                            #calls the auto iterator
            pass
        else:
            print('Parameter Error.')
    def Parameter_Checks(self): #checks for errors in the input parameters
        if (self.stored_randint_max-self.stored_randint_min+1)<self.stored_chromosome_length:
            self.bool_pcheck_safe=False
    def List_Iterator(self): #Auto Iterator
        for i in trange(self.stored_epoch_count,desc='Epoch'):
            if self.index_list==[] or self.bool_stop==False:                                                             #if no fit gene is found
                self.generation_list.append(self.Gene_List_Cross(input_list=self.generation_list[i],pop_limit=self.stored_pop_limit,
                                                                 crossover_method=self.preferred_crossover_method,crossover_prune_method=self.preferred_crossover_prune_method,
                                                                 verbose=self.bool_verbose,mutate=self.bool_mutate))    #generate new generation
                self.Gene_List_Eval(self.generation_list[i],evaluator=self.preferred_evaluator,verbose=self.bool_verbose,stop=self.bool_stop)                        #evaluate the new generation
                if self.index_list!=[]:                                                         #if fit gene found
                    self.Gene_List_Viewer(self.generation_list[i],self.bool_verbose)                           #display fit gene
                    return                                                                     #end loop
    def Gene_List_Gen(self,pop_limit=GLOBAL_POP_LIMIT,input_list=[], #Gene List Generator [Initial] [Standalone+Iterator]
                      chromosome_length=GLOBAL_CHROMOSOME_LENGTH,randint_min=GLOBAL_RANDINT_MIN,randint_max=GLOBAL_RANDINT_MAX,
                      gene_generator='random',evaluator='default-evaluator',
                      initial_eval=False): 
        result=[]
        temp=[]
        match gene_generator:
            case 'random':
                for i in trange(pop_limit,desc='Generating initial genes [Random]'):
                    temp=[]
                    for j in range(pop_limit):
                        temp.append(Gene(fitting_data=self.fitting_data,
                                        chromosome_length=chromosome_length,randint_min=randint_min,randint_max=randint_max,
                                        gene_generator=gene_generator,evaluator=evaluator,
                                        initial_eval=initial_eval))
                    result.append(temp)
            case 'input':
                print('Work in Progress')
        return result
    def Gene_List_Cross(self,input_list,pop_limit=GLOBAL_POP_LIMIT, #Gene List Crossover Function [Standalone+Iterator]
                        crossover_method='cartesian-prod-random-uniform',crossover_prune_method='random-survivor',
                        verbose=True,mutate=True): 
        result=[]
        cross=[]
        temp=[]
        if verbose:
            match crossover_method:
                case 'cartesian-prod-random-uniform':
                    ct_prod_vb='Crossover function executing [Cartesian]'
                    for i in trange(len(input_list),desc=ct_prod_vb):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
                case 'equate-cartesian-prod-random-uniform':
                    eq_prod_vb='Crossover function executing [Cartesian Equate]'
                    for i in trange(len(input_list),desc=eq_prod_vb):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
                case 'average-integer':
                    avg_int_vb='Crossover function executing [Average]'
                    for i in trange(len(input_list),desc=avg_int_vb):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
            match crossover_prune_method:
                case 'random-survivor':
                    rand_verbose='Selecting random survivors'
                    surv_list=[]
                    for i in trange(pop_limit,desc=rand_verbose):
                        temp=[]
                        for j in range(pop_limit):
                            surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                            surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                            temp.append(cross[surv_x][surv_y])
                        surv_list.append(temp)
                    result=surv_list
            return result
        else:
            match crossover_method:
                case 'cartesian-prod-random-uniform':
                    for i in range(len(input_list)):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
                case 'equate-cartesian-prod-random-uniform':
                    for i in range(len(input_list)):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
                case 'average-integer':
                    for i in range(len(input_list)):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
            match crossover_prune_method:
                case 'random-survivor':
                    surv_list=[]
                    for i in range(pop_limit):
                        temp=[]
                        for j in range(pop_limit):
                            surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                            surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                            temp.append(cross[surv_x][surv_y])
                        surv_list.append(temp)
                    result=surv_list
            return result
    def Gene_List_Eval(self,input_list, #Gene List Fitness Evaluation Function [Standalone+Iterator]
                       evaluator='default-evaluator',
                       verbose=True,stop=False): 
        result=[]
        temp=[]
        match evaluator:
            case 'default-evaluator':
                if verbose:
                    eval_str='Evaluating genes'
                    for i in trange(len(input_list),desc=eval_str):
                        for j in range(len(input_list[0])):
                            if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                                temp=[]
                                temp.append(i)
                                temp.append(j)
                                result.append(temp)
                                if stop:
                                    self.index_list=result
                                    return
                    self.index_list=result
                    pass
                else:
                    for i in range(len(input_list)):
                        for j in range(len(input_list[0])):
                            if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                                temp=[]
                                temp.append(i)
                                temp.append(j)
                                result.append(temp)
                                if stop:
                                    self.index_list=result
                                    return
                    self.index_list=result
                    pass
    def Gene_List_Viewer(self,input_list,verbose=True): #Gene List Viewer [Standalone+Iterator Version]
        if verbose:
            for i in tqdm(self.index_list,desc='Displaying genes'):
                print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())
        else:
            for i in self.index_list:
                print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())
    def GLCEF(self,input_list,stop=False): #Gene List Crossover & Evaluation Function [Iterator Version] [Generation Generator] [Deprecated]
        result=[]
        temp=[]
        for i in trange(len(input_list),desc='Crossover & Evaluation function executing'):
            for j in range(len(input_list)):
                temp=[]
                for k in range(len(input_list)):
                    for l in range(len(input_list)):
                        child=input_list[i][j].crossover_with_gene(input_list[k][l])
                        temp.append(child)
                        child.fitness_evaluation(self.fitting_data)
                        if child.fitness_factor==0:
                            temp2=[]
                            temp2.append(i)
                            temp2.append(j)
                            self.index_list.append(temp2)
                            if stop:
                                pass
                                
                result.append(temp)
        return result
    def Gene_List_Cross_I(self,input_list,verbose,crossover_prune_method,pop_limit,mutate): #Gene List Crossover Function [Iterator Version] [Deprecating]
        result=[]
        cross=[]
        temp=[]
        if verbose:
            cross_verbose='Crossover function executing'
            rand_verbose='Selecting random survivors'
            for i in trange(len(input_list),desc=cross_verbose):
                for j in range(len(input_list)):
                    temp=[]
                    for k in range(len(input_list)):
                        for l in range(len(input_list)):
                            temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate))
                    cross.append(temp)
            if crossover_prune_method=='random-survivor':
                surv_list=[]
                for i in trange(pop_limit,desc=rand_verbose):
                    temp=[]
                    for j in range(pop_limit):
                        surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                        surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                        temp.append(cross[surv_x][surv_y])
                    surv_list.append(temp)
                result=surv_list
            return result
        else: #verbose=False
            for i in range(len(input_list)):
                for j in range(len(input_list)):
                    temp=[]
                    for k in range(len(input_list)):
                        for l in range(len(input_list)):
                            temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate))
                    cross.append(temp)
            if crossover_prune_method=='random-survivor':
                surv_list=[]
                for i in range(pop_limit):
                    temp=[]
                    for j in range(pop_limit):
                        surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                        surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                        temp.append(cross[surv_x][surv_y])
                    surv_list.append(temp)
                result=surv_list
            return result
    def Gene_List_Eval_I(self,input_list,verbose,stop): #Gene List Fitness Evaluation Function [Iterator Version] [Deprecating]
        result=[]
        temp=[]
        if verbose:
            eval_str='Evaluating genes'
            for i in trange(len(input_list),desc=eval_str):
                for j in range(len(input_list[0])):
                    if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                        temp=[]
                        temp.append(i)
                        temp.append(j)
                        result.append(temp)
                        if stop:
                            self.index_list=result
                            return
            self.index_list=result
            pass
        else:
            for i in range(len(input_list)):
                for j in range(len(input_list[0])):
                    if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                        temp=[]
                        temp.append(i)
                        temp.append(j)
                        result.append(temp)
                        if stop:
                            self.index_list=result
                            return
            self.index_list=result
            pass

In [32]:
GList=Gene_List(manual=True,chromosome_length=1,randint_min=1,randint_max=10,pop_limit=4,mutate=False)
GList2=Gene_List(manual=True,chromosome_length=1,randint_min=-10,randint_max=10,pop_limit=4,mutate=False)

Generating initial genes [Random]: 100%|██████████| 4/4 [00:00<?, ?it/s]


In [33]:
GList3=Gene_List(manual=True,gene_generator='none')

In [34]:
GList3.generation_list.append(GList.Gene_List_Cross(GList2.generation_list[0],mutate=False,pop_limit=4,crossover_method='average-integer'))

Selecting random survivors: 100%|██████████| 4/4 [00:00<00:00, 4001.24it/s]


In [35]:
GList3.generation_list[1][0][0].chromosomes

[-2]

In [36]:
GList4=Gene_List(epoch=1000,stop=True)

Epoch:   2%|▏         | 23/1000 [00:04<03:05,  5.27it/s]

Gene [0, 6] Chromosomes: [0, 1, 2, 3] Fitness Factor: 0


<h6><i>Everything below is deprecated</i></h6>

In [37]:
def GLG(): #Gene List Generator [Initial]
    result=[]
    temp=[]
    for i in trange(GLOBAL_POP_LIMIT,desc='Generating genes'):
        temp=[]
        for j in range(GLOBAL_POP_LIMIT):
            temp.append(Gene(rand_gene=True))
        result.append(temp)
    return result

In [38]:
def GLCF(input_list): #Gene List Crossover Function
    result=[]
    temp=[]
    for i in trange(len(input_list),desc='Crossover function executing'):
        for j in range(len(input_list)):
            temp=[]
            for k in range(len(input_list)):
                for l in range(len(input_list)):
                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l]))
            result.append(temp)
    return result

In [39]:
def GLFEF(input_list,fitting_data,stop=False): #Gene List Fitness Evaluation Function
    result=[]
    temp=[]
    for i in trange(len(input_list),desc='Evaluating genes'):
        for j in range(len(input_list[0])):
            if input_list[i][j].fitness_evaluation(fitting_data)==0:
                temp=[]
                temp.append(i)
                temp.append(j)
                result.append(temp)
                if stop:
                    return result
    return result

In [40]:
def Gene_List_Viewer(input_list,index_list): #Gene List Viewer
    for i in tqdm(index_list,desc='Displaying genes'):
        print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())

In [41]:
def GCLRS(input_list): #Gene Crossover List Random Sampler
    len_list=len(input_list)
    parent_1=random.randint(0,len_list-1)
    parent_2=random.randint(0,len_list-1)
    print('\nParent 1 ['+str(parent_1)+']:'+input_list[parent_1][parent_1].display())
    print('\nParent 2 ['+str(parent_2)+']:'+input_list[parent_1][parent_1].display())
    print('\nChild 1 (P1xP2): '+input_list[parent_1][parent_2].display())
    print('\nChild 2 (P2xP1): '+input_list[parent_2][parent_1].display())

In [42]:
fitting_data=[x for x in range(GLOBAL_CHROMOSOME_LENGTH)]
list_of_genes=GLG()
first_gen=GLCF(list_of_genes)
index_list=GLFEF(first_gen,fitting_data=fitting_data,stop=True)
Gene_List_Viewer(first_gen,index_list)

Generating genes:   0%|          | 0/16 [00:00<?, ?it/s]


TypeError: Gene.__init__() got an unexpected keyword argument 'rand_gene'

In [ ]:
run=True
gene_list=GLG()
generation_list=[]
generation_list.append(gene_list)
index_list=[]
while run==False:
    for i in trange(GLOBAL_EPOCH_COUNT):
        if index_list==[]:
            generation_list.append(GLCF(generation_list[i]))
            index_list=GLFEF(generation_list[i],fitting_data,stop=True)
        if index_list!=[]:
            Gene_List_Viewer(generation_list[i],index_list)
            run=False
            break
    run=False

In [ ]:
GCLRS(first_gen)

In [ ]:
class Gene_ID_Gen: #deprecated
    gene_id_tracker=0
    def generate(self):
        gene_id=self.gene_id_tracker+1
        self.gene_id_tracker=self.gene_id_tracker+1
        return gene_id
    def latest(self):
        return self.gene_id_tracker

ID_Gen=Gene_ID_Gen()